# In-depth Analysis of Successful Negotiation Cases

View complete details of successful negotiations, including:
- Complete conversation transcript
- Deal Terms reached
- Memory (M) state
- Plan (P) strategy
- Negotiation process analysis

## 1. Setup and Imports

In [3]:
import sqlite3
import pandas as pd
import json
from datetime import datetime
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

print("✅ Import successful")

✅ Import successful


## 2. Connect to Database

In [4]:
# Modify the path here
DB_PATH = "negotiations (1).db"  # Change to your database path

conn = sqlite3.connect(DB_PATH)
print(f"✅ Connected to database: {DB_PATH}")

✅ Connected to database: negotiations (1).db


## 3. List All Successful Negotiations

In [5]:
# Get all successful negotiations
query = """
SELECT 
    session_id,
    student_name,
    scenario_name,
    student_role,
    ai_model,
    use_memory,
    use_plan,
    created_at,
    feedback_text TEXT
FROM negotiation_sessions
WHERE deal_reached = 1
ORDER BY created_at DESC
"""

successful_df = pd.read_sql_query(query, conn)

print(f"\n📊 Found {len(successful_df)} successful negotiations\n")

# Display list
display_df = successful_df.copy()
display_df['session_id_short'] = display_df['session_id'].str[:8]
display_df['M+P'] = display_df.apply(
    lambda x: '✅' if x['use_memory'] and x['use_plan'] else '❌', 
    axis=1
)

display(display_df[['session_id_short', 'student_name', 'scenario_name', 
                     'ai_model', 'M+P', 'created_at']])

print("\n💡 Copy session_id_short to view details")


📊 Found 11 successful negotiations



,session_id_short,student_name,scenario_name,ai_model,M+P,created_at
0,6bcbd0fe,Chris,Main_Street,openai/gpt-5,❌,2025-12-01T06:59:07.400156
1,03b5791c,Chris,Main_Street,openai/gpt-5,✅,2025-12-01T06:54:53.616963
2,06eb1797,Chris,Top_talent,openai/gpt-5,❌,2025-12-01T06:54:01.765574
3,d9e5ab85,Chris,Top_talent,openai/gpt-5,❌,2025-12-01T06:53:49.801746
4,2d6e0616,Chris,Main_Street,openai/gpt-5,❌,2025-12-01T06:17:38.355369
5,30c3c80c,Chris,Main_Street,openai/gpt-5,❌,2025-12-01T06:17:23.112858
6,746dbd3e,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:57.375665
7,05a9b9d7,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:51.933084
8,c5ab236a,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:49.456787
9,21d35936,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:41.849373



💡 Copy session_id_short to view details


## 4. View Specific Case Details

### 👇 Enter the session_id you want to view below (first 8 characters are sufficient)

In [21]:
# ============================================
# 👉 MODIFY HERE: Enter the session_id you want to view
# ============================================
SESSION_ID = "03b5791c"  # Change to the actual session_id (first 8 characters)

# If only first 8 characters, find the complete ID
if len(SESSION_ID) == 8:
    full_id_query = f"""
    SELECT session_id FROM negotiation_sessions 
    WHERE session_id LIKE '{SESSION_ID}%' AND deal_reached = 1
    """
    result = pd.read_sql_query(full_id_query, conn)
    if len(result) > 0:
        SESSION_ID = result['session_id'].iloc[0]
        print(f"✅ Found complete ID: {SESSION_ID}")
    else:
        print(f"❌ No successful cases found starting with {SESSION_ID} ")

# Get complete data
query = f"""
SELECT * FROM negotiation_sessions
WHERE session_id = '{SESSION_ID}'
"""

case_df = pd.read_sql_query(query, conn)

if len(case_df) == 0:
    print(f"❌ session_id not found: {SESSION_ID}")
else:
    case = case_df.iloc[0]
    print(f"\n✅ Case loaded: {SESSION_ID}")

✅ Found complete ID: 03b5791c-d8c7-480f-9810-db0f6653bd45

✅ Case loaded: 03b5791c-d8c7-480f-9810-db0f6653bd45


## 5. 📋 Case Overview

In [22]:
if len(case_df) > 0:
    print("=" * 80)
    print("                          Case Overview")
    print("=" * 80)
    
    print(f"\n🆔 Session ID:     {case['session_id']}")
    print(f"👤 Student:           {case['student_name']} (ID: {case['student_id']})")
    print(f"🎯 Scenario:           {case['scenario_name']}")
    print(f"🎭 Student Role:       {case['student_role']}")
    print(f"🤖 AI Role:         {case['ai_role']}")
    print(f"🧠 AI Model:         {case['ai_model']}")
    print(f"🔧 Memory Enabled:     {'Yes ✅' if case['use_memory'] else 'No ❌'}")
    print(f"📋 Plan Enabled:       {'Yes ✅' if case['use_plan'] else 'No ❌'}")
    print(f"🎲 Who Starts First:       {'student' if case['student_goes_first'] else 'AI'}")
    print(f"🔄 Total Rounds:       {case['total_rounds']}")
    
    # 计算时长
    try:
        start = pd.to_datetime(case['created_at'])
        end = pd.to_datetime(case['updated_at'])
        duration = (end - start).total_seconds() / 60
        print(f"⏳ Duration:           {duration:.1f} minutes")
    except:
        pass
    
    # 消息统计
    try:
        transcript = json.loads(case['transcript'])
        print(f"💬 Total Messages:       {len(transcript)}")
        print(f"📅 Start Time:       {case['created_at']}")
        print(f"✅ Completion Time:       {case['updated_at']}")
    except:
        pass

                          Case Overview

🆔 Session ID:     03b5791c-d8c7-480f-9810-db0f6653bd45
👤 Student:           Chris (ID: 123)
🎯 Scenario:           Main_Street
🎭 Student Role:       side2
🤖 AI Role:         side1
🧠 AI Model:         openai/gpt-5
🔧 Memory Enabled:     Yes ✅
📋 Plan Enabled:       Yes ✅
🎲 Who Starts First:       student
🔄 Total Rounds:       6
⏳ Duration:           36.3 minutes
💬 Total Messages:       13
📅 Start Time:       2025-12-01T06:54:53.616963
✅ Completion Time:       2025-12-01T07:31:09.857781


## 6. 🤝 Deal Terms Reached

In [23]:
if len(case_df) > 0:
    print("=" * 80)
    print("                          Deal Terms")
    print("=" * 80)
    
    print("\n📝 Student's Proposed Deal Terms:")
    print("-" * 80)
    if case['student_deal_json']:
        try:
            student_deal = json.loads(case['student_deal_json'])
            print(json.dumps(student_deal, indent=2, ensure_ascii=False))
        except Exception as e:
            print(f"Parse error: {e}")
    else:
        print("(Not recorded)")
    
    print("\n\n🤖 AI Confirmed Deal Terms:")
    print("-" * 80)
    if case['ai_deal_json']:
        try:
            ai_deal = json.loads(case['ai_deal_json'])
            print(json.dumps(ai_deal, indent=2, ensure_ascii=False))
        except Exception as e:
            print(f"Parse error: {e}")
    else:
        print("(Not recorded)")
    
    # 对比差异
    if case['student_deal_json'] and case['ai_deal_json']:
        try:
            student_deal = json.loads(case['student_deal_json'])
            ai_deal = json.loads(case['ai_deal_json'])
            
            print("\n\n🔍 Comparison Analysis:")
            print("-" * 80)
            
            all_keys = set(student_deal.keys()) | set(ai_deal.keys())
            
            for key in sorted(all_keys):
                student_val = student_deal.get(key, "(Missing)")
                ai_val = ai_deal.get(key, "(Missing)")
                
                if student_val == ai_val:
                    print(f"✅ {key:30s}: {student_val}")
                else:
                    print(f"⚠️  {key:30s}:")
                    print(f"   Student: {student_val}")
                    print(f"   AI:   {ai_val}")
        except:
            pass

                          Deal Terms

📝 Student's Proposed Deal Terms:
--------------------------------------------------------------------------------
{
  "final_price": 628000
}


🤖 AI Confirmed Deal Terms:
--------------------------------------------------------------------------------
{
  "final_price": 628000
}


🔍 Comparison Analysis:
--------------------------------------------------------------------------------
✅ final_price                   : 628000


In [24]:
case["ai_memory_history"]

'[{"round": "1.2", "content": "OFFERS: [Us: price: not yet; Them: price: 700,000 [unchanged]; Them-best-for-us: price: 700,000]\\nOPPONENT PATTERNS: [Opened with 700,000 anchor; no concessions yet; emphasized intrinsic value vs. town slump; invited our position (\\u201cWhere are you?\\u201d)]\\nOPPONENT PRIORITIES: [EXPLICIT: simplify life by selling 123 Main; EXPLICIT: price should reflect perceived true value, not discounted for downturn; Hypothesis: may value a clean/quick transaction]\\nOPPONENT CONSTRAINTS: [None stated; no explicit red lines beyond resisting \\u201csoft patch\\u201d discounts]"}, {"round": "2.2", "content": "OFFERS: [Us: price: not yet\\u2192590,000 [updated]; Them: price: 700,000\\u2192645,000 [updated]; Them-best-for-us: price: 645,000]\\nOPPONENT PATTERNS: [Conceded 700k\\u2192645k tied to speed/certainty (645k for fast/clean); Rejected our retrofit/soft-market discount framing; Did not accept 590k; Asked us to improve; Claims ability to close quickly; Cited o

In [25]:
case["ai_plan_history"]

'[{"round": "1.2", "content": "- ROUND GOAL: Reset the anchor to \\u2264600,000; surface urgency; provoke a counter below 700,000 without conceding.\\n- KEY LEVERS: Price (primary); framing of speed/clean close; seller\\u2019s eagerness to simplify.\\n- TACTICS: Anchor at 590,000 with concise rationale (market softness, rental comps ~500\\u2013550, conversion/capex needs); signal we can move quickly without overexplaining; ask 1\\u20132 probes on timing/other interest; avoid revealing adjacency/synergy; reference alternative sites to maintain leverage; commit to no movement above 620,000 this round.\\n- OFFER SCAFFOLD: price: 590,000"}, {"round": "2.2", "content": "- ROUND GOAL: Move seller off 645k; land acceptance at 610k or elicit a counter \\u2264635k; set up a 620\\u2013630k close next round if needed.\\n- KEY LEVERS: Price; speed/clean close (proof of funds, 10\\u201314 day timeline); challenge vague \\u201cother interest\\u201d; willingness to walk from the 640s.\\n- TACTICS: Ma

## 7. 🧠 Memory (M) State

In [17]:
if len(case_df) > 0:
    print("=" * 80)
    print("                     AI Memory (Final State)")
    print("=" * 80)
    
    if not case['use_memory']:
        print("\n❌ Memory not enabled for this case")
    elif not case['ai_memory']:
        print("\n(Memory state not recorded)")
    else:
        print("\n🧠 AI Memory recorded the following information:\n")
        print(case['ai_memory'])
        
        # 分析Memory内容
        memory_text = case['ai_memory']
        print("\n\n📊 Memory Analysis:")
        print("-" * 80)
        print(f"Character count: {len(memory_text)}")
        print(f"Line count: {len(memory_text.split('\n'))}")
        
        # 检查关键词
        keywords = ['OFFERS', 'PATTERNS', 'PRIORITIES', 'CONSTRAINTS']
        found = [kw for kw in keywords if kw in memory_text]
        print(f"Key sections included: {', '.join(found)}")

                     AI Memory (Final State)

❌ Memory not enabled for this case


## 8. 📋 Plan (P) Strategy

In [10]:
if len(case_df) > 0:
    print("=" * 80)
    print("                   AI Strategic Plan (Final State)")
    print("=" * 80)
    
    if not case['use_plan']:
        print("\n❌ Plan not enabled for this case")
    elif not case['ai_plan']:
        print("\n(Plan state not recorded)")
    else:
        print("\n📋 AI's Final Strategic Plan:\n")
        print(case['ai_plan'])
        
        # 分析Plan内容
        plan_text = case['ai_plan']
        print("\n\n📊 Plan Analysis:")
        print("-" * 80)
        print(f"Character count: {len(plan_text)}")
        print(f"Line count: {len(plan_text.split('\n'))}")
        
        # 检查关键词
        keywords = ['ROUND GOAL', 'KEY LEVERS', 'TACTICS', 'OFFER SCAFFOLD', 'RISK']
        found = [kw for kw in keywords if kw in plan_text]
        print(f"Strategy sections included: {', '.join(found)}")

                   AI Strategic Plan (Final State)

❌ Plan not enabled for this case


## 9. 💬 Complete Conversation Transcript（Transcript）

In [18]:
if len(case_df) > 0:
    print("=" * 80)
    print("                         Complete Conversation Transcript")
    print("=" * 80)
    
    try:
        transcript = json.loads(case['transcript'])
        total_messages = len(transcript)
        
        print(f"\n💬 共 {total_messages} 条消息\n")
        print("=" * 80)
        
        for i, message in enumerate(transcript, 1):
            # 解析消息格式: "Round X.Y - Label: content"
            if " - " in message:
                parts = message.split(" - ", 1)
                round_info = parts[0]
                
                if len(parts) > 1 and ":" in parts[1]:
                    speaker_content = parts[1].split(":", 1)
                    speaker = speaker_content[0].strip()
                    content = speaker_content[1].strip() if len(speaker_content) > 1 else ""
                    
                    # 判断是学生还是AI
                    if case['student_role'] in speaker or "Student" in speaker:
                        icon = "👤"
                        print(f"\n{icon} [{round_info}] {speaker}:")
                    else:
                        icon = "🤖"
                        print(f"\n{icon} [{round_info}] {speaker}:")
                    
                    # 打印内容（处理长消息）
                    if len(content) > 500:
                        print(f"   {content[:2000]}...")
                        print(f"   ... (共 {len(content)} characters)")
                    else:
                        print(f"   {content}")
                else:
                    print(f"\n💬 [{round_info}]")
                    print(f"   {parts[1] if len(parts) > 1 else message}")
            else:
                print(f"\n💬 消息 {i}:")
                if len(message) > 500:
                    print(f"   {message[:500]}...")
                    print(f"   ... (共 {len(message)} characters)")
                else:
                    print(f"   {message}")
            
            print("-" * 80)
        
    except Exception as e:
        print(f"\n❌ 解析transcript出错: {e}")

                         Complete Conversation Transcript

💬 共 8 条消息


🤖 [Round 1.1] The Candidate:
   Great to hear the offer is coming together—I’m genuinely excited about C&C and the chance to learn under Blair. To make this a win on both sides, here’s a package that would set me up to hit the ground running and deliver quickly:

- Work Location: Philadelphia as my home office
- Blair Rotation: Yes—3 months in DC under Blair to start, then return to the main office
- Start Date: August 12 (I can do any date August 1 or later to align with the Blair rotation)
- Base Salary: $180,000
- Remote Work: In-person, 0 days/week
- Signing Bonus: $40,000
- Vacation: 4 weeks

This reflects how I’ll contribute best: fully in-person with strong mentorship, a crisp start after Aug 1 to ensure the DC rotation kicks off right, and comp aligned with market and the scope of this role.

How does this land with you, and where do you have the most flexibility? If there are constraints, I’m open to explor

In [106]:
case['feedback_text']

'═══════════════════════════════════════\n📊 PERFORMANCE SUMMARY\n═══════════════════════════════════════\n\n- Final outcome: 31,000 points achieved\n- Overall assessment: You secured a deal above your BATNA of 0 points, which is a positive result under time constraints. However, you left significant value on the table by conceding on high-point issues early and not pushing harder on key financial terms.\n\n═══════════════════════════════════════\n🎯 STRATEGIC ANALYSIS\n═══════════════════════════════════════\n\n- **Start Date (15,000 points achieved out of 15,000 possible)**: You secured the maximum value by holding firm on August 1st or later. Your consistent emphasis on personal timing needs paid off as the recruiter conceded early. This was a strong tactical move, ensuring a high-value win without trade-offs.\n  \n- **Work Location (0 points achieved out of 4,000 possible)**: You conceded to New Haven (0 points) despite Philadelphia offering 4,000 points. While you articulated person

## 10. 📊 Negotiation Process Analysis

In [58]:
if len(case_df) > 0:
    print("=" * 80)
    print("                        Negotiation Process Analysis")
    print("=" * 80)
    
    try:
        transcript = json.loads(case['transcript'])
        total_messages = len(transcript)
        
        # 统计消息长度
        message_lengths = []
        student_messages = 0
        ai_messages = 0
        deal_signals = 0
        
        for msg in transcript:
            # 计算消息长度
            if " - " in msg:
                parts = msg.split(" - ", 1)
                if len(parts) > 1:
                    msg_content = parts[1]
                    message_lengths.append(len(msg_content))
            else:
                message_lengths.append(len(msg))
            
            # 统计发言者
            if case['student_role'] in msg:
                student_messages += 1
            elif case['ai_role'] in msg:
                ai_messages += 1
            
            # 检查deal信号
            if "$DEAL_REACHED$" in msg:
                deal_signals += 1
        
        print("\n📊 Basic Statistics:")
        print("-" * 80)
        print(f"Total Messages:        {total_messages}")
        print(f"Student Messages:        {student_messages}")
        print(f"AI Messages:          {ai_messages}")
        print(f"完成Round数:      {total_messages // 2}")
        print(f"$DEAL_REACHED$:  出现 {deal_signals} 次")
        
        if message_lengths:
            print("\n📏 Message Complexity:")
            print("-" * 80)
            print(f"Average Message Length:    {sum(message_lengths)/len(message_lengths):.0f} characters")
            print(f"Shortest Message:        {min(message_lengths)} characters")
            print(f"Longest Message:        {max(message_lengths)} characters")
        
        # 计算每Round平均时长
        try:
            start = pd.to_datetime(case['created_at'])
            end = pd.to_datetime(case['updated_at'])
            total_minutes = (end - start).total_seconds() / 60
            rounds = total_messages // 2
            
            if rounds > 0:
                print("\n⏱️  Time Analysis:")
                print("-" * 80)
                print(f"Total Duration:          {total_minutes:.1f} minutes")
                print(f"每Round平均:      {total_minutes/rounds:.1f} minutes")
                print(f"Average Per Message:    {total_minutes/total_messages:.1f} minutes")
        except:
            pass
        
        # 判断谈判风格
        print("\n🎯 Negotiation Characteristics:")
        print("-" * 80)
        
        avg_length = sum(message_lengths)/len(message_lengths) if message_lengths else 0
        if avg_length > 500:
            print("💬 Message Style:      Detailed (longer messages)")
        elif avg_length > 200:
            print("💬 Message Style:      Moderate")
        else:
            print("💬 Message Style:      Concise (shorter messages)")
        
        if total_minutes / rounds < 3 if rounds > 0 else False:
            print("⚡ Negotiation Pace:      快速（<3minutes/Round）")
        elif total_minutes / rounds < 5 if rounds > 0 else False:
            print("⚡ Negotiation Pace:      Normal（3-5minutes/Round）")
        else:
            print("⚡ Negotiation Pace:      缓慢（>5minutes/Round）")
        
        if rounds < case['total_rounds'] / 2:
            print("🎉 Deal Efficiency:      高（不到一半Round就达成）")
        elif rounds < case['total_rounds']:
            print("🎉 Deal Efficiency:      Normal")
        else:
            print("🎉 Deal Efficiency:      压线（用完所有Round）")
        
    except Exception as e:
        print(f"\n❌ Analysis error: {e}")

## 11. 🔍 Quick View: Comparison of Recent 3 Successful Cases

In [19]:
# 获取最近3个成功案例
recent_query = """
SELECT 
    session_id,
    student_name,
    scenario_name,
    ai_model,
    use_memory,
    use_plan,
    transcript,
    created_at,
    updated_at
FROM negotiation_sessions
WHERE deal_reached = 1
ORDER BY created_at DESC
LIMIT 3
"""

recent_df = pd.read_sql_query(recent_query, conn)

if len(recent_df) > 0:
    print("=" * 80)
    print("                 Comparison of Recent 3 Successful Cases")
    print("=" * 80)
    
    comparison_data = []
    
    for idx, row in recent_df.iterrows():
        try:
            transcript = json.loads(row['transcript'])
            msg_count = len(transcript)
            
            start = pd.to_datetime(row['created_at'])
            end = pd.to_datetime(row['updated_at'])
            duration = (end - start).total_seconds() / 60
            
            comparison_data.append({
                'Session': row['session_id'][:8],
                'Student': row['student_name'],
                'Scenario': row['scenario_name'],
                'Model': row['ai_model'].split('/')[-1],
                'M+P': '✅' if row['use_memory'] and row['use_plan'] else '❌',
                'Messages': msg_count,
                'Duration(min)': f"{duration:.1f}",
                'Rounds': msg_count // 2
            })
        except:
            pass
    
    if comparison_data:
        comparison_df = pd.DataFrame(comparison_data)
        display(comparison_df)
else:
    print("\n❌ No successful cases found")

                 Comparison of Recent 3 Successful Cases


,Session,Student,Scenario,Model,M+P,Messages,Duration(min),Rounds
0,6bcbd0fe,Chris,Main_Street,gpt-5,❌,10,26.1,5
1,03b5791c,Chris,Main_Street,gpt-5,✅,13,36.3,6
2,06eb1797,Chris,Top_talent,gpt-5,❌,8,23.0,4


## 12. Close Database Connection

In [13]:
conn.close()
print("✅ Database connection closed")

✅ Database connection closed
